<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/space_travel_demo_deepseek_EARTH_TO_MOON.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install colab-env --upgrade -q
!pip install unsloth -q

In [ ]:
import os
import colab_env
import json

Mounted at /content/gdrive


## Earth to Moon

In [ ]:
import os
import torch
from typing import List, Dict
from datetime import datetime, timedelta
from unsloth import FastLanguageModel

# Assume a global variable for the loaded model and tokenizer
deepseek_model = None
deepseek_tokenizer = None

def load_deepseek_model():
    """Loads the DeepSeek model and tokenizer using Unsloth."""
    global deepseek_model, deepseek_tokenizer

    if deepseek_model is None or deepseek_tokenizer is None:
        max_seq_length = 4096
        dtype = torch.bfloat16 if torch.cuda.is_bf16_supported() else torch.float16
        load_in_4bit = True

        print("Loading DeepSeek model...")
        deepseek_model, deepseek_tokenizer = FastLanguageModel.from_pretrained(
            model_name="unsloth/DeepSeek-R1-Distill-Llama-8B",
            max_seq_length=max_seq_length,
            dtype=dtype,
            load_in_4bit=load_in_4bit,
        )
        print("DeepSeek model loaded successfully.")

# Define a function to interact with the LLM (now DeepSeek)
def generate_flight_plan_with_llm(prompt: str, system_prompt: str = None, model: str = "deepseek", temperature=0.7, max_tokens=2048):
    """Generates a response from the DeepSeek LLM."""
    load_deepseek_model()

    if system_prompt:
        full_prompt = f"### System:\n{system_prompt}\n\n### User:\n{prompt}\n\n### Assistant:\n"
    else:
        full_prompt = f"### User:\n{prompt}\n\n### Assistant:\n"

    inputs = deepseek_tokenizer(full_prompt, return_tensors="pt").to("cuda")

    outputs = deepseek_model.generate(
        **inputs,
        max_new_tokens=max_tokens,
        temperature=temperature,
        do_sample=True if temperature > 0 else False,
        pad_token_id=deepseek_tokenizer.eos_token_id,
    )

    response_text = deepseek_tokenizer.decode(outputs[0][inputs["input_ids"].shape[1]:], skip_special_tokens=True)
    return response_text.strip()

def generate_flight_plan_iteratively(prompt: str, system_prompt: str = None, model: str = "deepseek", temperature=0.7, max_tokens_per_iteration=500):
    """Generates a flight plan iteratively from the DeepSeek LLM."""
    load_deepseek_model()

    full_response = ""
    current_prompt_text = prompt

    while True:
        if system_prompt:
            full_input_text = f"### System:\n{system_prompt}\n\n### User:\n{current_prompt_text}\n\n### Assistant:\n"
        else:
            full_input_text = f"### User:\n{current_prompt_text}\n\n### Assistant:\n"

        inputs = deepseek_tokenizer(full_input_text, return_tensors="pt").to("cuda")

        outputs = deepseek_model.generate(
            **inputs,
            max_new_tokens=max_tokens_per_iteration,
            temperature=temperature,
            do_sample=True if temperature > 0 else False,
            pad_token_id=deepseek_tokenizer.eos_token_id,
        )

        chunk = deepseek_tokenizer.decode(outputs[0][inputs["input_ids"].shape[1]:], skip_special_tokens=True)
        full_response += chunk

        if deepseek_tokenizer.eos_token_id in outputs[0]:
             break
        elif len(outputs[0]) - inputs["input_ids"].shape[1] < max_tokens_per_iteration:
            break

        current_prompt_text = f"{chunk} Continue:"
    return full_response.strip()


class SpaceFlightPlanningAgent:
    def __init__(self, llm_models: List[str]):
        self.llm_models = llm_models

    def generate_flight_plan(
        self,
        origin: str,
        destination: str,
        spacecraft: str,
        mission_objectives: List[str],
        constraints: List[str],
    ) -> Dict[str, Dict[str, str]]:
        system_prompt = f"""
You are a helpful and informative space flight planner.
You are tasked with generating a detailed flight plan for a mission from {origin} to {destination}
using a {spacecraft}. The mission objectives are: {mission_objectives}. The flight plan
must satisfy the following constraints: {constraints}.
Consider these factors:
Orbital mechanics (including Hohmann transfer orbits, delta-v calculations, and
apogee/perigee for Mars; translunar injection, lunar orbit insertion, and descent/ascent for Moon)
Spacecraft capabilities (delta-v, thrust-to-weight ratio, specific impulse)
Gravitational forces of relevant celestial bodies (Sun, Earth, Mars, Moon, etc.)
Radiation exposure (minimize exposure by considering solar activity and trajectory)
Communication windows (ensure contact with Earth throughout the mission)
Launch opportunities (identify optimal launch windows based on planetary/lunar alignment)
"""

        flight_plan_sections = {}

        if destination == "Mars":
            flight_plan_sections = {
                "Launch Date and Time": """Provide a **specific** launch date and time in UTC, considering
Do not provide a range of dates or suggest future windows. Choose a date
starting from 2025-07-03 20:27:07 UTC.
""",
                "Trajectory": {
                    "initial_prompt": "Provide a summary of the key phases and maneuvers of the trajectory.",
                    "follow_up_prompts": [
                        "Describe the launch process, including the type of launch vehicle, the target orbit, and the purpose of the initial orbit.",
                        "Explain the Hohmann transfer maneuver, including the delta-v requirements, the timing, and the resulting trajectory.",
                        "Detail the interplanetary cruise phase, including the duration, the communication strategy, and any planned course corrections.",
                        "Describe the approach to Mars and the Mars orbit insertion maneuver, including the delta-v requirements and the target orbit.",
                    ],
                },
                "Maneuver Schedule": "Detail each maneuver with its purpose, delta-v requirement, and timing.",
                "Communication Plan": "Explain how communication with Earth will be maintained throughout the mission, including communication systems, delays, and potential blackouts.",
                "Contingency Plans": "Describe contingency plans for various scenarios, such as launch vehicle failure, missed maneuver opportunities, spacecraft malfunctions, radiation exposure, and crew illness or injury.",
            }
        elif destination == "Moon":
            flight_plan_sections = {
                "Launch Date and Time": f"""Provide a **specific** launch date and time in UTC, considering
Do not provide a range of dates or suggest future windows. Choose a date
starting from {datetime.utcnow().strftime('%Y-%m-%d %H:%M:%S UTC')}.
""",
                "Trajectory": {
                    "Launch": "Describe the launch process, including the launch vehicle, launch site, and parking orbit.",
                    "Earth Departure and Translunar Injection": "Describe the Earth departure and translunar injection maneuvers, including delta-v requirements and burn duration.",
                    "Lunar Orbit Insertion": "Describe the lunar orbit insertion maneuver, including the burn duration, delta-v requirements, and target lunar orbit.",
                    "Lunar Descent and Landing": "Describe the lunar descent and landing process, including landing site selection, descent stages, and final landing procedures.",
                    "Lunar Surface Operations": "Describe the planned activities on the lunar surface, including scientific experiments, sample collection, and instrument setup.",
                    "Lunar Ascent and Return": "Describe the lunar ascent and return trajectory, including lunar ascent, lunar orbit rendezvous, trans-Earth injection, and coast to Earth.",
                    "Earth Re-entry and Landing": "Describe the Earth re-entry and landing process, including re-entry angle, heat shield, descent stages, and final landing/splashdown.",
                },
                "Maneuver Schedule": "Detail each maneuver with its purpose, delta-v requirement, and timing.",
                "Communication Plan": "Explain how communication with Earth will be maintained throughout the mission, including communication systems, delays, and potential blackouts.",
                "Contingency Plans": "Describe contingency plans for various scenarios, such as launch vehicle failure, missed maneuver opportunities, spacecraft malfunctions, radiation exposure, and crew illness or injury.",
            }

        all_flight_plans = {}

        for model in self.llm_models:
            flight_plan = {}
            # Define temperature here to be accessible within the loop
            temperature = 0.7 # Default temperature for generation

            for section_name, section_instructions in flight_plan_sections.items():
                if section_name == "Trajectory":
                    if destination == "Mars":
                        if isinstance(section_instructions, dict) and "initial_prompt" in section_instructions:
                            prompt = section_instructions["initial_prompt"]
                            trajectory_details = [generate_flight_plan_with_llm(prompt, system_prompt, "deepseek", temperature)]
                            for sub_question in section_instructions["follow_up_prompts"]:
                                prompt = f"""
Specifically, focus on the {section_name}:
{sub_question}
"""
                                trajectory_details.append(generate_flight_plan_with_llm(prompt, system_prompt, "deepseek", temperature))
                            flight_plan[section_name] = "\n".join(trajectory_details)
                    elif destination == "Moon":
                        trajectory_details = ""
                        for subsection_name, subsection_instructions in section_instructions.items():
                            prompt = f"""
Specifically, focus on the {subsection_name} of the Trajectory:
{subsection_instructions}
"""
                            response = generate_flight_plan_iteratively(prompt, system_prompt, "deepseek", temperature)
                            trajectory_details += f"\n**{subsection_name}:**\n{response}\n"
                        flight_plan[section_name] = trajectory_details
                else:
                    prompt = f"""
Specifically, focus on the {section_name}:
{section_instructions}
"""
                    response = generate_flight_plan_with_llm(prompt, system_prompt, "deepseek", temperature)

                    retry_count = 0
                    temp = temperature # Use the initial temperature for retries
                    # A more robust check for incomplete responses could be implemented here
                    # For example, checking if the response ends abruptly or lacks key information
                    while len(response) < 100 and retry_count < 3: # Simple length check for incompleteness
                        print(f"Response for {section_name} was truncated or incomplete. Retrying with temperature {temp}...")
                        response = generate_flight_plan_with_llm(prompt, system_prompt, "deepseek", temp)
                        retry_count += 1
                        temp += 0.1
                        if retry_count >= 3:
                            break

                    flight_plan[section_name] = response
            all_flight_plans[model] = flight_plan

        return all_flight_plans


In [ ]:
if __name__ == "__main__":
    # Ensure your CUDA environment is set up correctly for Unsloth's FastLanguageModel.
    # This code assumes a GPU is available.

    # Initialize agent to use DeepSeek LLM
    agent = SpaceFlightPlanningAgent(["deepseek"])

    # --- Earth to Moon Mission (DeepSeek LLM) ---
    print("\n--- Earth to Moon Mission (DeepSeek LLM) ---")
    print("Starting Moon flight plan generation...")
    all_flight_plans_moon = agent.generate_flight_plan(
        origin="Earth",
        destination="Moon",
        spacecraft="Orion spacecraft",
        mission_objectives=["Land on the Moon", "Conduct scientific experiments"],
        constraints=["Minimize radiation exposure", "Ensure crew safety"]
    )

    print("\n--- Moon flight plan generation complete. ---")
    print('\n')

    for model, flight_plan in all_flight_plans_moon.items():
        print(f"Flight plan generated by {model}:")
        print('\n')
        for section_name, section_content in flight_plan.items():
            print(f"{section_name}:\n{section_content}\n")


--- Earth to Moon Mission (DeepSeek LLM) ---
Starting Moon flight plan generation...

--- Moon flight plan generation complete. ---


Flight plan generated by deepseek:


Launch Date and Time:
Alright, so I need to figure out the optimal launch date and time for the mission to the Moon using the Orion spacecraft. The mission has two main objectives: landing on the Moon and conducting scientific experiments. The constraints are to minimize radiation exposure and ensure crew safety. 

First, I should consider the launch window. Launch opportunities are based on planetary and lunar alignment. Since we're targeting the Moon, the launch should occur when both the Earth and Moon are in favorable positions. Also, considering the communication windows, we need to ensure that once the spacecraft is launched, there's continuous contact with Earth throughout the mission.

Radiation exposure is a big concern. The mission must minimize this, so the trajectory should avoid periods of high solar act